Selenium approach

In [25]:
import requests
from lxml import html
import os
import sys
import time 
from collections import Counter
import pandas as pd
import re

In [26]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [49]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver')

In [50]:
driver.get("https://www.thechairmansbao.com/my-account/edit-account/")
#enter account info

Collect all links from collection of TCB articles

In [28]:
#Intialize main landing page
driver.get("https://www.thechairmansbao.com/")

all_links = []
elems = set(driver.find_elements(By.XPATH,"//a[@href]"))
for elem in elems:
    
    url = elem.get_attribute("href")
    all_links.append(url)

In [29]:
#add links from all subsequent pages
pages = ['https://www.thechairmansbao.com/?paged='+str(i) for i in range(2,436)]

In [30]:
for page in pages:
    print(page)
    driver.get(page)
    
    elems = set(driver.find_elements(By.XPATH,"//a[@href]"))
    for elem in elems:
        url = elem.get_attribute("href")
        all_links.append(url)
    


https://www.thechairmansbao.com/?paged=2
https://www.thechairmansbao.com/?paged=3
https://www.thechairmansbao.com/?paged=4
https://www.thechairmansbao.com/?paged=5
https://www.thechairmansbao.com/?paged=6
https://www.thechairmansbao.com/?paged=7
https://www.thechairmansbao.com/?paged=8
https://www.thechairmansbao.com/?paged=9
https://www.thechairmansbao.com/?paged=10
https://www.thechairmansbao.com/?paged=11
https://www.thechairmansbao.com/?paged=12
https://www.thechairmansbao.com/?paged=13
https://www.thechairmansbao.com/?paged=14
https://www.thechairmansbao.com/?paged=15
https://www.thechairmansbao.com/?paged=16
https://www.thechairmansbao.com/?paged=17
https://www.thechairmansbao.com/?paged=18
https://www.thechairmansbao.com/?paged=19
https://www.thechairmansbao.com/?paged=20
https://www.thechairmansbao.com/?paged=21
https://www.thechairmansbao.com/?paged=22
https://www.thechairmansbao.com/?paged=23
https://www.thechairmansbao.com/?paged=24
https://www.thechairmansbao.com/?paged=25


https://www.thechairmansbao.com/?paged=195
https://www.thechairmansbao.com/?paged=196
https://www.thechairmansbao.com/?paged=197
https://www.thechairmansbao.com/?paged=198
https://www.thechairmansbao.com/?paged=199
https://www.thechairmansbao.com/?paged=200
https://www.thechairmansbao.com/?paged=201
https://www.thechairmansbao.com/?paged=202
https://www.thechairmansbao.com/?paged=203
https://www.thechairmansbao.com/?paged=204
https://www.thechairmansbao.com/?paged=205
https://www.thechairmansbao.com/?paged=206
https://www.thechairmansbao.com/?paged=207
https://www.thechairmansbao.com/?paged=208
https://www.thechairmansbao.com/?paged=209
https://www.thechairmansbao.com/?paged=210
https://www.thechairmansbao.com/?paged=211
https://www.thechairmansbao.com/?paged=212
https://www.thechairmansbao.com/?paged=213
https://www.thechairmansbao.com/?paged=214
https://www.thechairmansbao.com/?paged=215
https://www.thechairmansbao.com/?paged=216
https://www.thechairmansbao.com/?paged=217
https://www

https://www.thechairmansbao.com/?paged=386
https://www.thechairmansbao.com/?paged=387
https://www.thechairmansbao.com/?paged=388
https://www.thechairmansbao.com/?paged=389
https://www.thechairmansbao.com/?paged=390
https://www.thechairmansbao.com/?paged=391
https://www.thechairmansbao.com/?paged=392
https://www.thechairmansbao.com/?paged=393
https://www.thechairmansbao.com/?paged=394
https://www.thechairmansbao.com/?paged=395
https://www.thechairmansbao.com/?paged=396
https://www.thechairmansbao.com/?paged=397
https://www.thechairmansbao.com/?paged=398
https://www.thechairmansbao.com/?paged=399
https://www.thechairmansbao.com/?paged=400
https://www.thechairmansbao.com/?paged=401
https://www.thechairmansbao.com/?paged=402
https://www.thechairmansbao.com/?paged=403
https://www.thechairmansbao.com/?paged=404
https://www.thechairmansbao.com/?paged=405
https://www.thechairmansbao.com/?paged=406
https://www.thechairmansbao.com/?paged=407
https://www.thechairmansbao.com/?paged=408
https://www

In [31]:
all_urls = list(set(all_links))
len(all_urls) 
all_urls = [page for page in all_urls if not bool(re.search('/page/',page))]
all_urls = [page for page in all_urls if not bool(re.search('paged=',page))]

In [42]:
len(all_urls)

9178

In [44]:
#save list of all unique links (may or may not actually be articles)
with open("/Users/elyebliss/Desktop/Thesis/thesis_updated/all_TCB_links.txt",'w') as outfile:
    for elem in all_urls:
        outfile.write(str(elem)+'\n')

Variables to collect:
-title
-url
-HSK level
-body text

In [51]:
all_title = []
all_url = []
all_HSK = []
all_body = []
ignore_list = []
counter = 0

In [53]:
for i in range(500,len(all_urls)): #
    
    
    try:
        elem = all_urls[i]
        
        driver.get(elem)
        time.sleep(5)
        
        #collect variables
        title = driver.title
        url = driver.current_url
        article_text =driver.find_element(By.XPATH, "/html/body").text
        HSK = re.findall('HSK\s[0-9]\+*',article_text)[0]
        all_ch = [item.start() for item in re.finditer(r'[\u4e00-\u9fff]',article_text)]
        body = article_text[all_ch[0]:all_ch[len(all_ch)-1]+1]
        body = body.replace('\n','')
        
        all_title.append(title)
        all_url.append(url)
        all_HSK.append(HSK)
        all_body.append(body)
        
        counter +=1
        print("link no: "+str(counter)+" worked")
    except:
        ignore_list.append(elem)
        
        counter +=1
        print("link no: "+str(counter)+" didn't work")

link no: 501 worked
link no: 502 worked
link no: 503 didn't work
link no: 504 worked
link no: 505 didn't work
link no: 506 worked
link no: 507 worked
link no: 508 worked
link no: 509 worked
link no: 510 worked
link no: 511 worked
link no: 512 worked
link no: 513 worked
link no: 514 worked
link no: 515 worked
link no: 516 worked
link no: 517 worked
link no: 518 worked
link no: 519 worked
link no: 520 worked
link no: 521 worked
link no: 522 worked
link no: 523 worked
link no: 524 worked
link no: 525 worked
link no: 526 worked
link no: 527 worked
link no: 528 worked
link no: 529 worked
link no: 530 worked
link no: 531 worked
link no: 532 worked
link no: 533 worked
link no: 534 worked
link no: 535 worked
link no: 536 worked
link no: 537 worked
link no: 538 worked
link no: 539 worked
link no: 540 worked
link no: 541 worked
link no: 542 didn't work
link no: 543 worked
link no: 544 worked
link no: 545 worked
link no: 546 worked
link no: 547 worked
link no: 548 worked
link no: 549 worked
link 

link no: 907 worked
link no: 908 worked
link no: 909 worked
link no: 910 worked
link no: 911 worked
link no: 912 worked
link no: 913 worked
link no: 914 worked
link no: 915 worked
link no: 916 worked
link no: 917 worked
link no: 918 worked
link no: 919 worked
link no: 920 worked
link no: 921 worked
link no: 922 worked
link no: 923 worked
link no: 924 worked
link no: 925 didn't work
link no: 926 worked
link no: 927 worked
link no: 928 worked
link no: 929 worked
link no: 930 worked
link no: 931 worked
link no: 932 worked
link no: 933 worked
link no: 934 worked
link no: 935 worked
link no: 936 worked
link no: 937 worked
link no: 938 worked
link no: 939 worked
link no: 940 worked
link no: 941 worked
link no: 942 worked
link no: 943 worked
link no: 944 worked
link no: 945 worked
link no: 946 worked
link no: 947 worked
link no: 948 worked
link no: 949 worked
link no: 950 worked
link no: 951 worked
link no: 952 worked
link no: 953 worked
link no: 954 worked
link no: 955 worked
link no: 956 wo

link no: 1299 worked
link no: 1300 worked
link no: 1301 didn't work
link no: 1302 worked
link no: 1303 worked
link no: 1304 worked
link no: 1305 worked
link no: 1306 worked
link no: 1307 worked
link no: 1308 worked
link no: 1309 worked
link no: 1310 worked
link no: 1311 worked
link no: 1312 worked
link no: 1313 worked
link no: 1314 worked
link no: 1315 worked
link no: 1316 worked
link no: 1317 worked
link no: 1318 worked
link no: 1319 worked
link no: 1320 worked
link no: 1321 worked
link no: 1322 worked
link no: 1323 worked
link no: 1324 didn't work
link no: 1325 worked
link no: 1326 worked
link no: 1327 worked
link no: 1328 worked
link no: 1329 worked
link no: 1330 worked
link no: 1331 worked
link no: 1332 worked
link no: 1333 worked
link no: 1334 worked
link no: 1335 worked
link no: 1336 worked
link no: 1337 didn't work
link no: 1338 worked
link no: 1339 didn't work
link no: 1340 worked
link no: 1341 worked
link no: 1342 worked
link no: 1343 worked
link no: 1344 worked
link no: 1345 

link no: 1685 worked
link no: 1686 worked
link no: 1687 worked
link no: 1688 worked
link no: 1689 worked
link no: 1690 worked
link no: 1691 worked
link no: 1692 worked
link no: 1693 worked
link no: 1694 worked
link no: 1695 worked
link no: 1696 worked
link no: 1697 worked
link no: 1698 worked
link no: 1699 worked
link no: 1700 worked
link no: 1701 worked
link no: 1702 worked
link no: 1703 worked
link no: 1704 worked
link no: 1705 worked
link no: 1706 worked
link no: 1707 worked
link no: 1708 worked
link no: 1709 worked
link no: 1710 worked
link no: 1711 worked
link no: 1712 worked
link no: 1713 worked
link no: 1714 worked
link no: 1715 worked
link no: 1716 worked
link no: 1717 worked
link no: 1718 worked
link no: 1719 worked
link no: 1720 worked
link no: 1721 worked
link no: 1722 worked
link no: 1723 worked
link no: 1724 worked
link no: 1725 worked
link no: 1726 worked
link no: 1727 worked
link no: 1728 worked
link no: 1729 worked
link no: 1730 worked
link no: 1731 worked
link no: 1732

link no: 2073 worked
link no: 2074 worked
link no: 2075 worked
link no: 2076 worked
link no: 2077 worked
link no: 2078 worked
link no: 2079 worked
link no: 2080 worked
link no: 2081 worked
link no: 2082 worked
link no: 2083 worked
link no: 2084 worked
link no: 2085 worked
link no: 2086 worked
link no: 2087 worked
link no: 2088 worked
link no: 2089 worked
link no: 2090 didn't work
link no: 2091 worked
link no: 2092 worked
link no: 2093 worked
link no: 2094 worked
link no: 2095 worked
link no: 2096 worked
link no: 2097 worked
link no: 2098 worked
link no: 2099 worked
link no: 2100 worked
link no: 2101 worked
link no: 2102 worked
link no: 2103 worked
link no: 2104 worked
link no: 2105 worked
link no: 2106 worked
link no: 2107 worked
link no: 2108 worked
link no: 2109 worked
link no: 2110 worked
link no: 2111 worked
link no: 2112 worked
link no: 2113 worked
link no: 2114 worked
link no: 2115 worked
link no: 2116 worked
link no: 2117 worked
link no: 2118 worked
link no: 2119 worked
link no:

...

link no: 7499 worked
link no: 7500 worked
link no: 7501 worked
link no: 7502 worked
link no: 7503 worked
link no: 7504 worked
link no: 7505 worked
link no: 7506 didn't work
link no: 7507 worked
link no: 7508 worked
link no: 7509 worked
link no: 7510 worked
link no: 7511 worked
link no: 7512 worked
link no: 7513 worked
link no: 7514 worked
link no: 7515 worked
link no: 7516 worked
link no: 7517 worked
link no: 7518 worked
link no: 7519 worked
link no: 7520 didn't work
link no: 7521 worked
link no: 7522 worked
link no: 7523 worked
link no: 7524 worked
link no: 7525 worked
link no: 7526 worked
link no: 7527 worked
link no: 7528 worked
link no: 7529 worked
link no: 7530 worked
link no: 7531 worked
link no: 7532 worked
link no: 7533 worked
link no: 7534 worked
link no: 7535 worked
link no: 7536 worked
link no: 7537 worked
link no: 7538 worked
link no: 7539 worked
link no: 7540 worked
link no: 7541 worked
link no: 7542 worked
link no: 7543 worked
link no: 7544 worked
link no: 7545 worked
lin

link no: 7887 worked
link no: 7888 worked
link no: 7889 worked
link no: 7890 worked
link no: 7891 worked
link no: 7892 worked
link no: 7893 worked
link no: 7894 worked
link no: 7895 worked
link no: 7896 worked
link no: 7897 worked
link no: 7898 worked
link no: 7899 worked
link no: 7900 worked
link no: 7901 worked
link no: 7902 worked
link no: 7903 worked
link no: 7904 worked
link no: 7905 worked
link no: 7906 worked
link no: 7907 worked
link no: 7908 worked
link no: 7909 worked
link no: 7910 worked
link no: 7911 worked
link no: 7912 worked
link no: 7913 worked
link no: 7914 worked
link no: 7915 worked
link no: 7916 worked
link no: 7917 worked
link no: 7918 didn't work
link no: 7919 didn't work
link no: 7920 worked
link no: 7921 worked
link no: 7922 worked
link no: 7923 worked
link no: 7924 worked
link no: 7925 worked
link no: 7926 worked
link no: 7927 worked
link no: 7928 worked
link no: 7929 worked
link no: 7930 worked
link no: 7931 worked
link no: 7932 worked
link no: 7933 worked
lin

link no: 8275 worked
link no: 8276 worked
link no: 8277 worked
link no: 8278 worked
link no: 8279 worked
link no: 8280 worked
link no: 8281 worked
link no: 8282 worked
link no: 8283 worked
link no: 8284 worked
link no: 8285 worked
link no: 8286 worked
link no: 8287 worked
link no: 8288 worked
link no: 8289 worked
link no: 8290 worked
link no: 8291 worked
link no: 8292 worked
link no: 8293 worked
link no: 8294 worked
link no: 8295 worked
link no: 8296 worked
link no: 8297 worked
link no: 8298 worked
link no: 8299 worked
link no: 8300 worked
link no: 8301 worked
link no: 8302 worked
link no: 8303 worked
link no: 8304 worked
link no: 8305 worked
link no: 8306 worked
link no: 8307 worked
link no: 8308 worked
link no: 8309 worked
link no: 8310 worked
link no: 8311 worked
link no: 8312 worked
link no: 8313 worked
link no: 8314 worked
link no: 8315 worked
link no: 8316 worked
link no: 8317 worked
link no: 8318 worked
link no: 8319 worked
link no: 8320 worked
link no: 8321 worked
link no: 8322

link no: 8662 worked
link no: 8663 worked
link no: 8664 worked
link no: 8665 worked
link no: 8666 worked
link no: 8667 worked
link no: 8668 worked
link no: 8669 worked
link no: 8670 worked
link no: 8671 worked
link no: 8672 worked
link no: 8673 worked
link no: 8674 worked
link no: 8675 worked
link no: 8676 worked
link no: 8677 worked
link no: 8678 worked
link no: 8679 worked
link no: 8680 worked
link no: 8681 worked
link no: 8682 worked
link no: 8683 worked
link no: 8684 worked
link no: 8685 worked
link no: 8686 worked
link no: 8687 worked
link no: 8688 worked
link no: 8689 worked
link no: 8690 worked
link no: 8691 worked
link no: 8692 worked
link no: 8693 worked
link no: 8694 worked
link no: 8695 worked
link no: 8696 worked
link no: 8697 worked
link no: 8698 worked
link no: 8699 worked
link no: 8700 worked
link no: 8701 didn't work
link no: 8702 worked
link no: 8703 worked
link no: 8704 worked
link no: 8705 worked
link no: 8706 worked
link no: 8707 worked
link no: 8708 worked
link no:

link no: 9050 worked
link no: 9051 worked
link no: 9052 worked
link no: 9053 worked
link no: 9054 worked
link no: 9055 worked
link no: 9056 worked
link no: 9057 worked
link no: 9058 worked
link no: 9059 worked
link no: 9060 worked
link no: 9061 worked
link no: 9062 worked
link no: 9063 worked
link no: 9064 worked
link no: 9065 worked
link no: 9066 worked
link no: 9067 worked
link no: 9068 worked
link no: 9069 worked
link no: 9070 worked
link no: 9071 worked
link no: 9072 worked
link no: 9073 worked
link no: 9074 worked
link no: 9075 worked
link no: 9076 worked
link no: 9077 worked
link no: 9078 worked
link no: 9079 worked
link no: 9080 worked
link no: 9081 worked
link no: 9082 worked
link no: 9083 worked
link no: 9084 worked
link no: 9085 worked
link no: 9086 worked
link no: 9087 worked
link no: 9088 worked
link no: 9089 worked
link no: 9090 worked
link no: 9091 worked
link no: 9092 worked
link no: 9093 worked
link no: 9094 worked
link no: 9095 worked
link no: 9096 worked
link no: 9097

In [54]:
#Save current output
TCB_df = pd.DataFrame(list(zip(all_HSK,all_title,all_url,all_body)),columns=\
                     ['label','title','url','text'])

In [55]:
len(TCB_df)

8887

Counter(TCB_df.label)

Counter({'HSK 5': 1565,
         'HSK 1': 1285,
         'HSK 3': 1865,
         'HSK 2': 1351,
         'HSK 4': 1849,
         'HSK 6': 825,
         'HSK 6+': 147})

Previous run had the below numbers, but might include errors:
Counter({'HSK 4': 2119,
         'HSK 1': 1485,
         'HSK 5': 1721,
         'HSK 2': 1519,
         'HSK 3': 2043})

In [89]:
with open('/Users/elyebliss/Desktop/Thesis/thesis_updated/TCB_df.csv','w') as outfile:
    TCB_df.to_csv(outfile,index=False)

In [83]:
with open('/Users/elyebliss/Desktop/Thesis/thesis_updated/TCB_df.csv','r') as infile:
    TCB_df = pd.read_csv(infile)

Clean text body of articles:

In [86]:
def clean_body(text):
    
    output = text
    
    remove_after = 'SMALL NORMAL LARGE XLSCRIPT:SIMPLIFIED TRADITIONALPINYIN:SHOW HIDESPACE:DEFAULT MEDIUM LARGE'

    if bool(re.search(remove_after,output)):
        
        output = re.sub('.*'+remove_after,'',output)
    
    return output

In [87]:
TCB_df.text = TCB_df.text.apply(clean_body)

Some HSK 1s (especially the "How to"s) are mostly English text. But not all. 

Some of these aren't actually HSKs... but have HSK links in below areas shown, so they still are marked in my database

In [88]:
TCB_df.text.iloc[10]

'北京时间12月20日，在NCAA橄榄球比赛中，何佩璋（JacksonHe）代表的亚利桑那州立大学70:7打败了亚利桑那大学。何佩璋更是达阵得分，成为了第一个在NCAA比赛中得分的中国人。何佩璋身高1.75米，100公斤，今年22岁。之前在12月6日和加州大学洛杉矶分校的比赛中，何佩璋第一次代表亚利桑那州立大学参加比赛，成为第一个参加NCAA比赛的中国球员。橄榄球是美国人最喜欢的运动，但在中国很少有人喜欢。去美国之前，何佩璋对橄榄球也不太了解。17岁时，何佩璋从广东到美国读高中，才慢慢开始喜欢这种运动，并且开始练习。2019年，他开始在亚利桑那州立大学学习。他也是从那时候开始参加美国橄榄球比赛的'

Issues:
- '加载中' might be because time is needed
- HSK6+ not captured by regex

If no '加载中' is found in body of df, and article is not HSK 6, I can keep it.

In [57]:
def find_error(text):
    
    if bool(re.search('加载中',text)):
        return True
    else:
        return False

In [58]:
download_error = TCB_df[TCB_df.text.apply(find_error)]

Clean output data base

In [4]:
with open('/Users/elyebliss/Desktop/Thesis/thesis_updated/TCB_df_final.csv','r') as infile:
    TCB_df = pd.read_csv(infile)

In [13]:
Counter(TCB_df.label)

Counter({'HSK 4': 2119,
         'HSK 1': 1485,
         'HSK 5': 1721,
         'HSK 2': 1519,
         'HSK 3': 2043})

Redo HSK 6 and 6+

In [14]:
#Intialize main landing page
driver.get("https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=1")

all_links = []
elems = set(driver.find_elements(By.XPATH,"//a[@href]"))
for elem in elems:
    
    url = elem.get_attribute("href")
    all_links.append(url)

In [16]:
#add links from all subsequent pages
pages = ['https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged='+str(i) for i in range(2,48)]

for page in pages:
    print(page)
    driver.get(page)
    
    elems = set(driver.find_elements(By.XPATH,"//a[@href]"))
    for elem in elems:
        url = elem.get_attribute("href")
        all_links.append(url)

https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=2
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=3
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=4
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=5
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=6
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=7
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=8
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=9
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=10
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=11
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=12
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=13
https://www.thechairmansbao.com/?text&low=6&high=7&from&to&read=All&paged=14
https:/

In [17]:



    
all_urls = list(set(all_links))
len(all_urls) #1332

1332

In [18]:
all_urls

['https://www.thechairmansbao.com/more-graduate-students-able-to-apply-for-shanghai-hukou/',
 'https://www.thechairmansbao.com/page/44/?text&low=6&high=7&from&to&read=All&paged=43',
 'https://www.thechairmansbao.com/researchers-suggest-asias-glaciers-could-shrink-by-13-by-2100-threatening-water-supply/',
 'https://www.thechairmansbao.com/second-hand-high-end-goods-increase-in-popularity-for-younger-generation/',
 'https://www.thechairmansbao.com/page/24/?text&low=6&high=7&from&to&read=All&paged=22',
 'https://www.thechairmansbao.com/hollywood-to-remake-low-budget-chinese-fantasy-movie/',
 'https://www.thechairmansbao.com/virus-hunter-keeps-watch-for-asias-next-deadly-threat/',
 'https://www.thechairmansbao.com/liu-damings-success-inspires-chinas-disabled-population/',
 'https://www.thechairmansbao.com/page/16/?text&low=6&high=7&from&to&read=All&paged=16#',
 'https://www.thechairmansbao.com/page/36/?text&low=6&high=7&from&to&read=All&paged=1',
 'https://www.thechairmansbao.com/greek-arc

313